In [1]:
folder = "sentences/"
filename = "summary-receta"
filetoread = '{}{}.txt'.format(folder,filename)
sentences_list = []
with open(filetoread, 'r', encoding='utf-8') as f:
    sentences_list = f.readlines()

In [2]:
import pandas as pd
import re
import numpy as np
import logging

In [3]:
logging.basicConfig(level=logging.ERROR)

In [4]:
df = pd.read_csv('verbs-all-new.csv', engine='python', encoding='utf-8')
df = df.replace(np.nan, '', regex=True)

In [5]:
df.head()

,root,indicative_present_2p_s,indicative_present_1p_s,indicative_present_1p_p,indicative_present_3p_s,indicative_present_3p_p,indicative_preterite_2p_s,indicative_preterite_1p_s,indicative_preterite_1p_p,indicative_preterite_3p_s,...,perfect_present_2p_s,perfect_present_1p_s,perfect_present_1p_p,perfect_present_3p_s,perfect_present_3p_p,perfect_past_2p_s,perfect_past_1p_s,perfect_past_1p_p,perfect_past_3p_s,perfect_past_3p_p
0,ser,eres,soy,somos,es,son,fuiste,fui,fuimos,fue,...,has sido,he sido,hemos sido,ha sido,han sido,habías sido,había sido,habíamos sido,había sido,habían sido
1,ver,ves,veo,vemos,ve,ven,viste,vi,vimos,vio,...,has visto,he visto,hemos visto,ha visto,han visto,habías visto,había visto,habíamos visto,había visto,habían visto
2,venir,vienes,vengo,venimos,viene,vienen,viniste,vine,vinimos,vino,...,has venido,he venido,hemos venido,ha venido,han venido,habías venido,había venido,habíamos venido,había venido,habían venido
3,ir,vas,voy,vamos,va,van,fuiste,fui,fuimos,fue,...,has ido,he ido,hemos ido,ha ido,han ido,habías ido,había ido,habíamos ido,había ido,habían ido
4,volver,vuelves,vuelvo,volvemos,vuelve,vuelven,volviste,volví,volvimos,volvió,...,has vuelto,he vuelto,hemos vuelto,ha vuelto,han vuelto,habías vuelto,había vuelto,habíamos vuelto,había vuelto,habían vuelto


In [6]:
df.iloc[6]

root                                     estar
indicative_present_2p_s                  estás
indicative_present_1p_s                  estoy
indicative_present_1p_p                estamos
indicative_present_3p_s                   está
indicative_present_3p_p                  están
indicative_preterite_2p_s            estuviste
indicative_preterite_1p_s               estuve
indicative_preterite_1p_p            estuvimos
indicative_preterite_3p_s               estuvo
indicative_preterite_3p_p           estuvieron
indicative_imperfect_2p_s              estabas
indicative_imperfect_1p_s               estaba
indicative_imperfect_1p_p            estábamos
indicative_imperfect_3p_s               estaba
indicative_imperfect_3p_p              estaban
indicative_conditional_2p_s           estarías
indicative_conditional_1p_s            estaría
indicative_conditional_1p_p         estaríamos
indicative_conditional_3p_s            estaría
indicative_conditional_3p_p           estarían
indicative_fu

In [7]:
def removeEmpty(lis):
    return list(filter(None, lis))

In [8]:
def fetchVerbs(tense, df):
    tense_list = df.filter(regex=("{}.*".format(tense))).to_numpy().flatten().tolist()
    tense_list = removeEmpty(tense_list)
    return tense_list

In [9]:
moods = ['indicative', 'perfect']
tenses = ['present', 'preterite', 'imperfect', 'conditional', 'future', 'past']

In [10]:
import itertools
names = list(map('_'.join, itertools.product(moods, tenses)))

In [11]:
for i, item in enumerate(names, start=0):
    print(i,item)

0 indicative_present
1 indicative_preterite
2 indicative_imperfect
3 indicative_conditional
4 indicative_future
5 indicative_past
6 perfect_present
7 perfect_preterite
8 perfect_imperfect
9 perfect_conditional
10 perfect_future
11 perfect_past


In [12]:
conjugationMap = {}
for i in names:
    conjugationMap[i] = fetchVerbs(i, df)

conjugationMap.keys()

dict_keys(['indicative_present', 'indicative_preterite', 'indicative_imperfect', 'indicative_conditional', 'indicative_future', 'indicative_past', 'perfect_present', 'perfect_preterite', 'perfect_imperfect', 'perfect_conditional', 'perfect_future', 'perfect_past'])

In [13]:
haber_present = df[df['root']=='haber'].iloc[:,1:6].to_numpy().flatten().tolist()
estar_present = fetchVerbs("indicative_present", df[df['root']=='estar'])
estar_preterite = fetchVerbs("indicative_preterite", df[df['root']=='estar'])
estar_imperfect = fetchVerbs("indicative_imperfect", df[df['root']=='estar'])

In [14]:
df = df[df['root']!='haber']

In [15]:
f = open('{}.html'.format(filename),'w')

In [16]:
notdone = [True]*len(sentences_list)

In [17]:
def doFormatting(sentence, start, end, color):
    sentence = sentence[:start] + "<b style=\"color:{};\">".format(color) + \
                   sentence[start:end] + "</b>" + sentence[end:]
    return sentence

In [18]:
def isDone(sentence):
    if "</b>" in sentence:
        return True
    return False

In [19]:
def findTense(verbList, sentence, color):
    formattedSentence = sentence
    for one_verb in verbList:
        a = re.search(r'\b({})\b'.format(one_verb), formattedSentence.lower(),  re.UNICODE)
        
        if (a is not None):
            logging.debug("Found match [%s] verb [%s]", a, one_verb)
            formattedSentence = doFormatting(formattedSentence, a.start(), a.end(), color)
            logging.debug("Formatted sentence [%s]", formattedSentence)
    return formattedSentence

In [20]:
import cssmaker
cssmaker.do()

In [21]:
tense, color = cssmaker.getTenseColor()

In [22]:
colormaphex = dict(zip(tense, color))
colormaphex

{'#pfuture': '#ffd9cc',
 '#pconditional': '#cccc99',
 '#pimperfect': '#ffff99',
 '#ppreterite': '#8cff66',
 '#ppresent': '#33ccff',
 '#sfuture': '#ff794d',
 '#sconditional': '#666633',
 '#simperfect': '#ffff00',
 '#spreterite': '#00ff00',
 '#spresent': '#0099cc',
 '#ifuture': '#cc3300',
 '#iconditional': '#333300',
 '#iimperfect': '#cccc00',
 '#ipreterite': '#339933',
 '#ipresent': '#003366'}

https://www.w3schools.com/colors/colors_names.asp

In [23]:
with open('tabl2.html', 'r') as file:
    message = file.read().replace('\n', '')

In [24]:
# sentences_list = ["Te estaba buscando "]

In [25]:
estar_preterite

['estuviste', 'estuve', 'estuvimos', 'estuvo', 'estuvieron']

In [26]:
for i in range(len(sentences_list)):
    one_sentence = sentences_list[i]
    # check for -ndo ending
    # progressive +ndo
    ending_in_ndo = re.search(r'\b(\w+)\s(\w+ndo)\b', one_sentence.lower(),  re.UNICODE)
    ending_in_ido = re.search(r'\b(\w+)\s(\w+ido)\b', one_sentence.lower(),  re.UNICODE)
    if ending_in_ndo is not None:
#         pass
        print("-----{} | {}".format(ending_in_ndo.group(1), ending_in_ndo.group(2)))
        #check the previous word
        if ending_in_ndo.group(1) in estar_present:
            one_sentence = doFormatting(one_sentence, ending_in_ndo.start(), ending_in_ndo.end(), \
                                        colormaphex["#ipresent"])
        elif ending_in_ndo.group(1) in estar_preterite:
            one_sentence = doFormatting(one_sentence, ending_in_ndo.start(), ending_in_ndo.end(), \
                                        colormaphex["#ipreterite"])

        elif ending_in_ndo.group(1) in estar_imperfect:
            one_sentence = doFormatting(one_sentence, ending_in_ndo.start(), ending_in_ndo.end(), \
                                        colormaphex["#iimperfect"])
#         else:
#             one_sentence = doFormatting(one_sentence, one_sentence.index(ending_in_ndo.group(2)),\
#                                       one_sentence.index(ending_in_ndo.group(2))+len(ending_in_ndo.group(2)) , \
#                                         dict_["indicative_progressive_imperfect"])
    
    
    
#     one_sentence = findTense(indicative_present_perfect, one_sentence, dict_["indicative_present_perfect"])
    one_sentence = findTense(conjugationMap["indicative_present"], one_sentence, colormaphex["#ipresent"])
    one_sentence = findTense(conjugationMap["indicative_preterite"], one_sentence, colormaphex["#ipreterite"])
    one_sentence = findTense(conjugationMap["indicative_imperfect"], one_sentence, colormaphex["#iimperfect"])
    one_sentence = findTense(conjugationMap["indicative_conditional"], one_sentence, colormaphex["#iconditional"])
    one_sentence = findTense(conjugationMap["indicative_future"], one_sentence, colormaphex["#ifuture"])
    one_sentence = findTense(conjugationMap["perfect_present"], one_sentence, colormaphex["#ppresent"])


    if ending_in_ido is not None:
         #check the previous word
        if ending_in_ido.group(1) in estar_present:
            pass
#             one_sentence = doFormatting(one_sentence, ending_in_ido.start(), ending_in_ido.end(), \
#                                         colormaphex["indicative_present_perfect"])
        elif ending_in_ido.group(1) in estar_preterite:
            pass
#             one_sentence = doFormatting(one_sentence, ending_in_ido.start(),ending_in_ido.end(), \
#                                         colormaphex["indicative_preterite"])

        elif ending_in_ido.group(1) in estar_imperfect:
            pass
#             one_sentence = doFormatting(one_sentence, ending_in_ido.start(), ending_in_ido.end(), \
#                                         colormaphex["indicative_imperfect"])

    
    formated = one_sentence + "<br/>"
    message += formated

-----semana | cocinando


In [27]:
message += "</p></body></html>"

In [28]:
f.write(message)
f.close()